In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib


In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import io, time, json
import requests
from bs4 import BeautifulSoup